In [69]:
import random
import NeuralNet as nn
import statistics as stats
import pandas as pd
#generates random population amount of random neural networks
#intialize with varying weights for each layer but the same amount
#of layers and nodes per layer
#initialized with random activation functions as well
#outputs neural nets as a list
def genRandom(population, nodes_per_layer):
    neuralNets=[]
    layers=len(nodes_per_layer)
    for p in range(0,population):
        activation_funcs=[]
        for q in range(0,layers-1):
            alpha=random.randint(0,2)
            if(alpha==2):
                activation_funcs.append(3)
            else:
                activation_funcs.append(alpha)
        net= nn.NeuralNet(activation_funcs,nodes_per_layer)
        neuralNets.append(net)
    return neuralNets


2

In [2]:


#Initial training of random population before genetic algs do their
#thing, pass in list of neural networks and the training dictionaries
#optionally pass in iterations for back propogation to be ran
#outputs updated neural nets 
def runBackProp(neuralNets, tics_to_inputs, tics_to_outputs, its=1):
    #get all the tics
    tics=tics_to_inputs.keys()
    #for each neural network
    for a in range(0,len(neuralNets)):
        #choose amount of iterations to initially train neural nets
        for b in range(0,its):
            #for each tic
            for b in tics:
                #back propogate over each tic
                inputs=tics_to_inputs[b]
                outputs=tics_to_inputs[b]
                neuralNets[a].backProp(inputs,outputs)
    return neuralNets

In [21]:

def meanAbsoluteDev(right, estimate):
    total=0
    div=len(right)
    if(div==0):
        return None
    for a in range(0,div):
        total+=abs(right[a]-estimate[a])
    return total/div

def medianAbsoluteDev(right, estimate):
    even=False
    listOfDiff=[]
    length= len(right)
    if(length==0):
        return None
    for a in range(length):
        listOfDiff.append(abs(right[a]-estimate[a]))
    listOfDiff=sorted(listOfDiff)
    if length % 2 == 0:
        val1= listOfDiff[int(length/2)]
        val2= listOfDiff[(int(length/2))-1]
        summ=val1+val2
        return summ/2
    else:
        index= length/2
        index= int(index-0.5)
        return listOfDiff[index]

def cheby(right, estimate):
    listOfDiff=[]
    for a in range(len(right)):
        listOfDiff.append(abs(right[a]-estimate[a]))
    return max(listOfDiff)


def leastSquares(right, estimate):
    listOfDiff=[]
    total=0
    for a in range(len(right)):
        partial= right[a]-estimate[a]
        square= partial*partial
        total+=square
    return total


#takes in the actual values and what the neural net outputs
#returns the fitness score for the neural net
def fitness(actualVals,output):
    mad= meanAbsoluteDev(actualVals, output)
    #med= medianAbsoluteDev(actualVals, output)
    #ls= 1/leastSquares(actualVals, output)
    #cheb= 1/cheby(actualVals, output)
    #sumOfAll=mad+med+ls+cheb
    #fitScore=mad+cheb+ls
    if(mad==None):
        return None
    mad=1/mad
    return mad





















In [4]:
#Pass in single neural network and training data dictionaries
#outputs a single fitness score for the neural net
def singleScore(neuralNet, tics_to_outputs, tics_to_inputs):
    scores=[]
    for tic in tics_to_outputs.keys():
        calc=[]
        real=[]
        for p in range(0,len(tics_to_inputs[tic])):
            if(p<100):
                calc.append(neuralNet.calculateOutput(tics_to_inputs[tic][p], single_input=True))
                real.append(tics_to_outputs[tic][p])
        if(fitness(real,calc)!=None):
            scores.append(fitness(real, calc))
    score=stats.mean(scores)
    return score

#Pass in list of neural networks and the training data dictionaries
#returns a dictionary of the neural networks and their scores 
def cumulativeScore(neuralNets, tics_to_outputs, tics_to_inputs):
    scoresDictionary={}
    for i in neuralNets:
        score=singleScore(i,tics_to_outputs, tics_to_inputs)
        scoresDictionary.update({i:score})
    return scoresDictionary

In [23]:
#returns the highest fitness score from the score dictionary
def findMostFit(scoresDictionary):
    scores=scoresDictionary.values()
    return max(scores)

#Takes a scores dictionary and normalizes them to values from
#0 to 1 and returns said dictionary
def normalizeScores(scoresDictionary):
    maxVal=findMostFit(scoresDictionary)
    #print(maxVal)
    #total=0;
    #i=0;
    for a in scoresDictionary.keys():
        score=scoresDictionary[a]
        #total+=score
        #i+=1
        norm=score/maxVal
        updater={a: norm}
        scoresDictionary.update(updater)
    #print(total/i)
    return scoresDictionary

In [6]:


#roulette wheel selection
#input the scores dictionary of neural nets with their scores
#and the amount of parents you want
#returns a dictionary of the to be parent neural nets and their fitness scores
def selection(scoresDictionary, amount):
    selection={}
    #normalize the scores
    scoresDictionary2=normalizeScores(scoresDictionary)
    #find sum of all the scores
    valTotal=sum(scoresDictionary2.values())
    #incrementer
    val=0
    #get keys 
    keys=scoresDictionary2.keys()
    #create list of tuples
    otherList=[]
    for a in keys:
        #get probability of reproduction
        prob=scoresDictionary2[a]/valTotal
        #multiply to 100 for simplicity
        prob=prob*100
        #add to val incrementer to get range of numbers
        prob=prob+val
        #create tuple
        tup=(a,prob)
        #update the list
        otherList.append(tup)
        #update the val incrementer
        val=prob
    #loop until we have all those selected to breed
    while(len(selection)<amount):
        #get a random val between 0 and 1 and multiply by 100
        randVal=random.random()*100
        for a in range(0,len(otherList)):
            #get the value
            val=otherList[a][1]
            if(val>randVal):
                break
        #get the original score and neural network
        keyToSee=otherList[a][0]
        valToSee=scoresDictionary[keyToSee]
        #add the selection
        selection.update({keyToSee: valToSee})
    return selection


In [7]:
#Turns the inputted one dimensional matrix to a three dimensional 
#matrix with nodes_per_layer dimensions
def turnOneToThree(listInOne, nodes_per_layer):
    threeDList=[]
    counter=0
    for a in range(0,len(nodes_per_layer)-1):
        twoDList=np.random.rand(nodes_per_layer[a],nodes_per_layer[a+1])
        for b in range(0, nodes_per_layer[a]):
            oneDList=np.zeros(nodes_per_layer[a+1])
            for c in range(0,nodes_per_layer[a+1]):
                oneDList[c]=listInOne[counter]
                counter+=1
            twoDList[b]=oneDList
        threeDList.append(twoDList)
    return threeDList


#Turns the inputted three dimensional matrix into a single list
def turnThreeToOne(matrix):
    listToReturn=[]
    for a in range(0,len(matrix)):
        for b in range(0,len(matrix[a])):
            for c in range(0, len(matrix[a][b])):
                listToReturn.append(matrix[a][b][c])
    return listToReturn

In [8]:

#This function takes in a single neural network
#Mutates its weights (not activation functions)
#and returns the mutated neural net
#inversion mutation
def mutateThatJohn(neuralNetToMutate, nodes_per_layer):
    #don't mutate the activation functions
    activation_funcs=neuralNetToMutate.activation_functions
    #creates new neural network
    neuralNetToReturn= nn.NeuralNet(activation_funcs,nodes_per_layer)
    weightsOf=neuralNetToMutate.weights
    #turn weights to one dimension
    weights1d= turnThreeToOne(weightsOf)
    #get total amount of weights
    totalAmtOfWeights=len(weights1d)
    b=0
    c=0
    #loop until we get two different indices
    while c==b:
        b= random.randint(0,totalAmtOfWeights)
        c= random.randint(0,totalAmtOfWeights)
    #grab sublist from those indices
    if b<c:
        subList=weights1d[b:c]
    else:
        subList=weights1d[c:b]
    #reverse the list
    subList.reverse()
    #replace elements in the list with reversed list
    if b<c:
        weights1d[b:c]=subList
    else:
        weights1d[c:b]=subList
    #need it back in three dimensions
    weightsBack=turnOneToThree(weights1d, nodes_per_layer)
    #set mutated weights to new neural net
    neuralNetToReturn.weights=weightsBack
    #return mutated neural net
    return neuralNetToReturn


In [9]:

#inputs two neural networks and a list of the nodes per layer 
#and crosses over the activation functions and weights of the
#two neural networks and returns the network that is their child
def crossover(neuralNetwork1, neuralNetwork2, nodes_per_layer):
    #get the weights of each neural network
    weights1=neuralNetwork1.weights
    weights2=neuralNetwork2.weights
    #get the activation functions of each neural network
    activation1=neuralNetwork1.activation_functions
    activation2=neuralNetwork2.activation_functions
    #get number of activation functions
    actives=len(activation1)
    #get random int for one-point crossover of activation functions
    cross=random.randint(0,actives)
    activation_funcs=[]
    for a in range(0,actives):
        if a<cross:
            activation_funcs.append(activation1[a])
        else:
            activation_funcs.append(activation2[a])
    #create initial neural net to return
    net= nn.NeuralNet(activation_funcs,nodes_per_layer)
    #get total amount of weights
    totalAmtOfWeights=0
    for a in range(0,len(weights1)):
        for b in range(0,len(weights1[a])):
            for c in range(0,len(weights1[a][b])):
                totalAmtOfWeights+=1
    #grab two random ints for two-point crossover
    b=0
    c=0
    while b==c:
        b= random.randint(0,totalAmtOfWeights)
        c= random.randint(0,totalAmtOfWeights)
    #find the lower index
    if b<c:
        first=b
        second=c
    else:
        first=c
        second=b
    counter=0
    #crossover weights to child
    for a in range(0,len(weights1)):
        for b in range(0,len(weights1[a])):
            for c in range(0,len(weights1[a][b])):
                if counter<first:
                    net.weights[a][b][c]=weights1[a][b][c]
                elif counter>=first and counter<second:
                    net.weights[a][b][c]=weights2[a][b][c]
                else:
                    net.weights[a][b][c]=weights1[a][b][c]
                counter+=1
    #return child
    return net

In [10]:

#Takes in a dictionary of those selected to be parents
#Takes in a dictionary of all the neural nets and their scores
#Takes in a population threshold (optionally takes in proportion to mutate)
#Returns list of mutated to pass to next generation
def mutateThoseJohns(population, selectionDictionary, scoresDictionary, nodes_per_layer, mutate=0.07):
    #get the list of neural networks
    listOfPotential=list(scoresDictionary.keys())
    #get the list of the selected neural networks
    listOfSelected=list(selectionDictionary.keys())
    #find the length of the list of potential
    totalLength=len(listOfPotential)
    #set amount selected to 0
    amountForMutate=0
    #set amount to select to the proportion of the population needed to
    #be mutated
    amountToMutate=population*mutate
    #list of mutated
    listOfMutated=[]
    #loop until all have been selected for mutation
    while(amountForMutate<amountToMutate):
        #get a random index to select
        randomIndex=random.randint(0,totalLength-1)
        #select it
        neuralNetToMutate=listOfPotential[randomIndex]
        #if it was not already selected
        if(neuralNetToMutate not in listOfSelected):
            #mutate it
            neuralNetToMutate=mutateThatJohn(neuralNetToMutate, nodes_per_layer)
            #add it to the list
            listOfMutated.append(neuralNetToMutate)
            #increment
            amountForMutate+=1
    #return list
    return listOfMutated

In [11]:

#generates next generation with population as the threshold
#value for amount in the next generation and scoresDictionary that
#maps neural nets to their fitness scores.
#Returns list of next generation of neural nets
def evaluation(population, selection, scoresDictionary, nodes_per_layer):
    #start list of next population
    nextPopulation=[]
    #grab the parents we will be using
    parents=list(selection.keys())
    #mutate some of the population and append them for the next generation
    nextPopulation.extend(mutateThoseJohns(population, selection, scoresDictionary, nodes_per_layer))
    #append parents to the next population
    nextPopulation.extend(parents)
    #get list of neural networks
    thelist1 = [(k, v) for k, v in scoresDictionary.items()] 
    #find the best neural net
    best=findMostFitNeuralNet(thelist1)
    nextPopulation.append(best[0])
    #loop until we have a full next population
    while len(nextPopulation)<population: 
        #parent indices
        par1=0
        par2=0
        #until two different parents
        while par1==par2:
            par1=random.randint(0,len(parents)-1)
            par2=random.randint(0,len(parents)-1)
        #grab the parents
        parent1=parents[par1]
        parent2=parents[par2]
        #create the child
        child= crossover(parent1, parent2, nodes_per_layer)
        #append the child
        nextPopulation.append(child)
    return nextPopulation


In [12]:

#Executes iterations worth of generations of neural networks
def letsDoSomeGeneticAlgorithms(nodes_per_layer, population, tics_to_outputs, tics_to_inputs, amount=0.25,iterations=60):
    #create initial population of neural networks
    neuralNets=genRandom(population, nodes_per_layer)
    #get amount of parents wanted
    parents=amount*population
    #set loop condition
    i=0
    #after iterations amount of iterations
    while i<iterations:
        #get the scores of the neural networks
        scoresDictionary=cumulativeScore(neuralNets, tics_to_outputs, tics_to_inputs)
        #grab parents for next generation
        selectionDictionary=selection(scoresDictionary, parents)
        #create next population
        nextPop=evaluation(population, selectionDictionary, scoresDictionary, nodes_per_layer)
        #set that to neuralNets
        neuralNets=nextPop
        #increment i
        i+=1
    #return the neural network loop
    return neuralNets


In [13]:
frame=pd.read_csv("stock_names.csv")

In [14]:
import pandas as pd
import numpy as np

def getInputs(tic, date, inputData): 
    listOfDates= list(inputData.index)
    listOfNums=[x for x in range(0,len(listOfDates))]
    datesToNums= dict(zip(listOfDates, listOfNums))
    new_dict = dict([(value, key) for key, value in datesToNums.items()]) 
    try:
        beginningIndex= datesToNums[date]
    except:
        return -1
    endIndex=beginningIndex-20
    if(endIndex<0):
        return -1
    relevantDates=[]
    for i in range(endIndex, beginningIndex):
        relevantDates.append(new_dict[i])
    output=[]
    for i in relevantDates:
        output.append(inputData['close'][i])
        output.append(inputData['range'][i])
        output.append(inputData['singleDay'][i])
        output.append(inputData['dayToDay'][i])
        if i == relevantDates[-1]:
            output.append(inputData['low'][i])
            output.append(inputData['high'][i])
            output.append(inputData['average'][i])
            output.append(inputData['open'][i])
            output.append(inputData['volume'][i])
            output.append(inputData['twelveDay'][i])
            output.append(inputData['twentySixDay'][i])
            output.append(inputData['volumeEMA'][i])
            output.append(inputData['fiftyTwoDayHigh'][i])
            output.append(inputData['fiftyTwoWeekHigh'][i])
            output.append(inputData['fiftyTwoDayLow'][i])
            output.append(inputData['fiftyTwoWeekLow'][i])
            output.append(inputData['fiftyTwoWeekAverage'][i])
            output.append(inputData['fiftyTwoDayStandDev'][i])
            output.append(inputData['fiftyTwoWeekStandDev'][i])
    return np.array(output)
    
def inputsForBackProp(tics):
    inputters=[]
    outputters=[]
    k=0
    for tic in tics:
        print(tics[k])
        k+=1
        tic=str(tic)
        output_values = pd.read_csv('training/' + tic + r'.csv')
        print(len(output_values))
        #make sure we have data here
        while(len(output_values)<1):
            alpha=getRandomTics(1)
            output_values=pd.read_csv('training/'+alpha[0]+r'.csv')
        #creates an array of input vectors for a given stock and the training days
        input_values = [0]*len(output_values.index)
        data = pd.read_csv('normalized_data/' + tic + r'.csv')
        data = data.set_index('date')
        i = 0
        for date in output_values.date:
            input = getInputs(tic,date,data)
            #catches error if not enough previous days
            if type(input) ==type(-1):
                output_values = output_values[output_values.date != date]
                input_values = input_values[:-1]
            else:
                input_values[i] = input
                i = i + 1
            

        inputters.append(input_values)
        outputters.append(output_values['dayToDay'].to_numpy())
    #map tics to their respective lists of inputs and outputs
    input_dict=dict(zip(tics, inputters))
    output_dict=dict(zip(tics,outputters))
    #return list of both dicts
    return [input_dict, output_dict]
def inputsForTesting(tics):
    inputters=[]
    outputters=[]
    k=0
    for tic in tics:
        print(tics[k])
        k+=1
        tic=str(tic)
        output_values = pd.read_csv('testing/' + tic + r'.csv')
        print(len(output_values))
        while(len(output_values)<1):
            alpha=getRandomTics(1)
            output_values=pd.read_csv('testing/'+alpha[0]+r'.csv')
        #creates an array of input vectors for a given stock and the training days
        input_values = [0]*len(output_values.index)
        data = pd.read_csv('normalized_data/' + tic + r'.csv')
        data = data.set_index('date')
        i = 0
        for date in output_values.date:
            input = getInputs(tic,date,data)
            #catches error if not enough previous days
            if type(input) ==type(-1):
                output_values = output_values[output_values.date != date]
                input_values = input_values[:-1]
            else:
                input_values[i] = input
                i = i + 1
            

        inputters.append(input_values)
        outputters.append(output_values['dayToDay'].to_numpy())
    #map tics to their respective lists of inputs and outputs
    input_dict=dict(zip(tics, inputters))
    output_dict=dict(zip(tics,outputters))
    #return list of both dicts
    return [input_dict, output_dict]
def findMostFitNeuralNet(thelist):
    maxer=0
    index=-1
    for a in range(0,len(thelist)):
        if(thelist[a][1]>maxer):
            maxer=thelist[a][1]
            index=a

    return thelist[index]
    

In [22]:
def directionPred(actual,predicted):
    right=0;
    wrong=0;
    for i in range(0,len(actual)):
        for j in range(0,len(actual[i])):
            if(predicted[i][j]>0 and actual[i][j]>0):
                right+=1
            elif(predicted[i][j]<0 and actual[i][j]<0):
                right+=1
            #elif(predicted[i][j]==0 and actual[i][j]==0):
                #right+=1
            else:
                wrong+=1
    return [right,wrong]
def averageError(actual, predicted):
    error=0
    ticker=0
    for i in range(0,len(actual)):
        for j in range(0,len(actual[i])):
            val=actual[i][j]-predicted[i][j]
            if(val<0):
                val=val*(-1)
            error+=val
            ticker+=1
    return error/ticker
def bestPredictions(best):
    actual=[]
    predicted=[]
    for tic in outputsForTesting[0].keys():
        calc=[]
        real=[]
        for p in range(0,len(outputsForTesting[0][tic])):
            calc.append(best[0].calculateOutput(outputsForTesting[0][tic][p], single_input=True))
            real.append(outputsForTesting[1][tic][p])
        actual.append(real)
        predicted.append(calc)
    return [actual, predicted]
from statistics import mean
def wholePopAveragePredictions(final):
    actual=[]
    predicted=[]
    for tic in outputsForTesting[0].keys():
        calc=[]
        real=[]
        for p in range(0,len(outputsForTesting[0][tic])):
            temp=[]
            for k in range(0,len(final)):
                temp.append(final[k].calculateOutput(outputsForTesting[0][tic][p], single_input=True))

            calc.append(mean(temp))
            real.append(outputsForTesting[1][tic][p])
        actual.append(real)
        predicted.append(calc)
    return [actual, predicted]


In [16]:
tics=[]
for i in range(0,len(frame['Ticker'])):
    tics.append(frame['Ticker'][i])
def getRandomTics(n):
    i=0
    ret=[]
    while(i<n):
        #get random index
        k=random.randint(0,len(tics)-1)
        ret.append(tics[k])
        i+=1
    return ret

In [17]:
#data setup
randos=getRandomTics(20)
outputs=inputsForBackProp(randos)
outputsForTesting=inputsForTesting(randos)


KBWB
1490
FTFT
1990
DOYU
96
RVT
3500
CPLP
2270
HYACW
289
CIBR
803
FLLCU
82
ZTR
3500
UFO
147
FDUSL
356
ESE
3500
IBUY
668
WHLM
3500
SBRA
3106
ICON
3500
ISEM
229
FBIO
1447
SGMO
3462
USX
283
KBWB
589
FTFT
921
DOYU
46
RVT
2431
CPLP
967
HYACW
108
CIBR
353
FLLCU
35
ZTR
2334
UFO
60
FDUSL
149
ESE
2247
IBUY
289
WHLM
1809
SBRA
1389
ICON
2299
ISEM
95
FBIO
620
SGMO
1528
USX
124


In [19]:
#Script to get results on finished neural net population
#Set the nodes per layer which implicitly sets the amount of layers
nodes_per_layer=[95,5,5,1]
#Get the final population of neural nets
final=letsDoSomeGeneticAlgorithms(nodes_per_layer, 60, outputs[1], outputs[0])
#find their fitness scores

44.74572894943695
44.74572894943695
44.74572894943695
49.782963884533665
73.94772829310584
73.94772829310584
83.41638306320554
83.41638306320554
148.55146685175941
159.52781372889058
159.52781372889058
159.52781372889058
159.52781372889058
159.52781372889058
160.2806068152685
171.4781377739353
171.4781377739353
171.4781377739353
171.4781377739353
171.4781377739353
171.4781377739353
171.4781377739353
171.4781377739353
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167704846744
186.89167

In [20]:

a=cumulativeScore(final, outputs[1], outputs[0])
#put this dictionary into a list of tuples
thelist1 = [(k, v) for k, v in a.items()] 
#find the best neural net
best=findMostFitNeuralNet(thelist1)
#Find predictions on the best one and the population as a whole
bestPreds=bestPredictions(best)
popPreds=wholePopAveragePredictions(final)
#Find Stats
bestStats=directionPred(bestPreds[0],bestPreds[1])
bestError=averageError(bestPreds[0],bestPreds[1])
avStats=directionPred(popPreds[0],popPreds[1])
avError=averageError(popPreds[0],popPreds[1])
#Report Stats
print("Here is how the most fit neural net faired on averaged: ")
print(bestError)
print("Here is how the most fit neural net faired directionally: ")
print("Right: ")
print(bestStats[0])
print("Wrong: ")
print(bestStats[1])
print("Here is how the neural net pop faired on averaged: ")
print(avError)
print("Here is how the neural net pop faired directionally: ")
print("Right: ")
print(avStats[0])
print("Wrong: ")
print(avStats[1])

Here is how the most fit neural net faired on averaged: 
0.029618342647213167
Here is how the most fit neural net faired directionally: 
Right: 
7856
Wrong: 
10423
Here is how the neural net pop faired on averaged: 
0.028882845419688243
Here is how the neural net pop faired directionally: 
Right: 
7818
Wrong: 
10461


(<NeuralNet.NeuralNet at 0x1e714156ac8>, 198.88878430025702)

In [115]:

for k in range(0, len(best[0].weights)):
    np.savetxt("{}new.csv".format(k),best[0].weights[k], delimiter=', ', fmt='%f')

In [76]:
bestStats=[]
bestError=[]
avStats=[]
avError=[]
for touchdown in range(0,30):
    #Script to get results on finished neural net population
    #Set the nodes per layer which implicitly sets the amount of layers
    nodes_per_layer=[95,1,1,1]
    #Get the final population of neural nets
    final=letsDoSomeGeneticAlgorithms(nodes_per_layer, 60, outputs[1], outputs[0])
    #find their fitness scores
    a=cumulativeScore(final, outputs[1], outputs[0])
    #put this dictionary into a list of tuples
    thelist1 = [(k, v) for k, v in a.items()] 
    #find the best neural net
    best=findMostFitNeuralNet(thelist1)
    #Find predictions on the best one and the population as a whole
    bestPreds=bestPredictions(best)
    popPreds=wholePopAveragePredictions(final)
    #Find Stats
    bestStats.append(directionPred(bestPreds[0],bestPreds[1]))
    bestError.append(averageError(bestPreds[0],bestPreds[1]))
    print(touchdown)
    print(bestError[touchdown])
    print(best[0].activation_functions)
    avStats.append(directionPred(popPreds[0],popPreds[1]))
    avError.append(averageError(popPreds[0],popPreds[1]))
#Report Stats
right=0
righto=0
for yes in range(0,len(bestStats)):
    right+=bestStats[yes][0]
for no in range(0,len(avStats)):
    righto+=avStats[no][0]
right=right/len(bestStats)
righto=righto/len(avStats)
print("Here is how the most fit neural net faired on averaged: ")
print(sum(bestError)/len(bestError))
print("Here is how the most fit neural net faired directionally: ")
print("Right: ")
print(right)
print("Here is how the neural net pop faired on averaged: ")
print(sum(avError)/len(avError))
print("Here is how the neural net pop faired directionally: ")
print("Right: ")
print(righto)

0
0.021303346883551396
[1, 1, 3]
1
0.021279616065666564
[0, 1, 0]
2
0.021278657860021578
[0, 3, 1]
3
0.021272900897696546
[0, 3, 3]
4
0.021272084664261337
[1, 0, 1]
5
0.02129495289698527
[0, 0, 3]
6
0.021270894649052227
[1, 1, 0]
7
0.02129361390169323
[1, 3, 0]
8
0.021278238895052114
[3, 1, 1]
9
0.021270778272537347
[1, 1, 3]
10
0.02129037004368542
[0, 1, 3]
11
0.021311310275228798
[3, 3, 1]
12
0.021272504620847645
[0, 0, 0]
13
0.02129449220790419
[0, 3, 3]
14
0.02127242150931598
[0, 3, 1]
15
0.021290487021799497
[1, 3, 1]
16
0.021288317527989445
[1, 0, 0]
17
0.021290489187046173
[0, 1, 1]
18
0.021310243382967498
[3, 0, 1]
19
0.021270489711816366
[0, 1, 3]
20
0.021270113668412766
[3, 1, 3]
21
0.02153159615088308
[1, 3, 3]
22
0.02137257179757528
[3, 1, 3]
23
0.021301626434111338
[0, 1, 3]
24
0.021274136326675658
[3, 0, 3]
25
0.021323123145638216
[3, 3, 0]
26
0.02127127913170506
[1, 0, 0]
27
0.021290153471811527
[0, 0, 1]
28
0.021291478105115866
[1, 0, 3]
29
0.021296309981944542
[1, 1, 3

In [71]:
from tempfile import TemporaryFile
def exportNeuralNet(best, name, leng):
    path='/NeuralNetworks/' + name + '.npy';
    np.savez(path,best[0].weights[0], best[0].weights[1], best[0].weights[2])

In [73]:
pen=0
z=0
for i in range(0,len(popPreds[1])):
    for j in range(0,len(popPreds[1][i])):
        pen+=1
        if(bestPreds[0][i][j]==0):
            z+=1
[pen,z]

[18279, 2624]

In [74]:
pen-z

15655

In [72]:
best[0].activation_functions

[1, 1, 1]